### to run this code, you need to have standalone EMBOSS (getorf), BLAST, and HMMER installed

In [2]:
import subprocess
import os
from glob import glob
from Bio import SearchIO
from Bio import SeqIO 
from Bio.Blast import NCBIXML 
from collections import defaultdict

## load and adjust all paths

In [3]:
outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta")
hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" # zde je hmm, musi existovat
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"

### Extract ORFs from genomes

In [45]:
def run_getorf(genome, orf_file):
    command = ["getorf", "-sequence", genome, "-outseq", orf_file, "-table", "0", "-minsize", "450", "-find", "1"] #adjust (120 aa, START to STOP, standard gen. code)
    subprocess.run(command, check=True)

outdir = "/home/majnusova/all/projects/plv/data/eustig_orfs/" #adjust
genomes = glob("/home/majnusova/all/projects/plv/data/genomes_test/*.fasta") #adjust - this is just a test dataset

for genome in genomes:
    orf_file = f"{outdir}{genome.split('/')[-1].replace('.fasta', '_orfs.fa')}"
    run_getorf(genome, orf_file)

Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)
Find and extract open reading frames (ORFs)


KeyboardInterrupt: 

### hmmsearch with HMM against those extracted ORFs -> hmmout

In [ ]:
# running hhmmsearch against multiple databases at once and saving the resulting files using the .hmmout suffix
def run_hmmsearch(orf_file, hmm_model, output_file):
    command = ["hmmsearch", "-o", output_file, hmm_model, orf_file]
    subprocess.run(command, check=True)

hmm_model = "/home/majnusova/all/projects/bilabrum/data/hmm/Packiging_ATPase.hmm" #adjust
output_directory = "/home/majnusova/all/projects/plv/data/eustigs_hmmout/" #adjust
orf_files = glob("/home/majnusova/all/projects/plv/data/eustig_orfs/*_orfs.fa") #adjust: list of orf databases


# iterate over the list of database files and run hmmsearch 
for orf_file in orf_files:
    output_file = f"{output_directory}{orf_file.split('/')[-1].replace('_orfs.fa', '_atp.hmmout')}" 
    run_hmmsearch(orf_file, hmm_model, output_file) #needs to be nested inside the loop!


### saving IDs of ALL homologs above inclusion threshold found by HMMER and name of the assembly they were found in
### genome_scaffold_ids - genome: ids

In [4]:
# names of viral orfs and the genome they originate from
genome_scaffold_orfs = {}

for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    genome_name = os.path.basename(file).split('_atp.hmmout')[0] # getting the real names of the genomes; os.path.basename(file) extracts the base name from the file's full path.
    if genome_name not in genome_scaffold_orfs:
        genome_scaffold_orfs[genome_name] = []

    hmmer_file = SearchIO.read(file, "hmmer3-text")
    
    for record in hmmer_file:
        if record.is_included:
            scaffold_id = record.id  # scaffold ID - removing _orf
            genome_scaffold_orfs[genome_name].append(scaffold_id)
            
total_viral_orfs = sum(len(i)for i in genome_scaffold_orfs.values())
total_viral_orfs

280

In [5]:
genome_scaffold_orfs

{'Chic10_scaffolds': ['NODE_5080_length_5696_cov_26.6876_1',
  'NODE_381_length_27838_cov_14.2215_25',
  'NODE_7293_length_4272_cov_22.9291_3',
  'NODE_5397_length_5445_cov_37.4879_2',
  'NODE_6066_length_4954_cov_14.3046_4',
  'NODE_847_length_17049_cov_12.9829_3',
  'NODE_7343_length_4249_cov_22.2454_2',
  'NODE_19673_length_1550_cov_34.6107_1',
  'NODE_1409_length_12728_cov_15.5927_2',
  'NODE_1574_length_11923_cov_14.5027_1',
  'NODE_711_length_18631_cov_14.7806_4',
  'NODE_15445_length_2031_cov_23.9084_2',
  'NODE_15112_length_2083_cov_16.6144_1',
  'NODE_12470_length_2567_cov_17.4088_2',
  'NODE_34064_length_837_cov_18.9655_1',
  'NODE_4057_length_6629_cov_27.157_1',
  'NODE_865_length_16875_cov_12.6059_4',
  'NODE_869_length_16855_cov_12.4798_4',
  'NODE_6477_length_4696_cov_21.0899_1',
  'NODE_17368_length_1781_cov_15.3621_1',
  'NODE_18418_length_1671_cov_21.013_1',
  'NODE_352_length_29307_cov_12.7034_4',
  'NODE_31105_length_924_cov_16.8895_1',
  'NODE_4114_length_6570_cov_1

In [6]:
# creating a dict of lists (genomes: ids)
genome_scaffold_ids = {}

for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    genome_name = os.path.basename(file).split('_atp.hmmout')[0] # getting the real names of the genomes; os.path.basename(file) extracts the base name from the file's full path.
    if genome_name not in genome_scaffold_ids:
        genome_scaffold_ids[genome_name] = []

    hmmer_file = SearchIO.read(file, "hmmer3-text")
    
    for record in hmmer_file:
        if record.is_included:
            scaffold_id = record.id.rsplit("_", 1)[0] # scaffold ID - removing _orf
            genome_scaffold_ids[genome_name].append(scaffold_id)

In [7]:
genome_scaffold_ids

{'Chic10_scaffolds': ['NODE_5080_length_5696_cov_26.6876',
  'NODE_381_length_27838_cov_14.2215',
  'NODE_7293_length_4272_cov_22.9291',
  'NODE_5397_length_5445_cov_37.4879',
  'NODE_6066_length_4954_cov_14.3046',
  'NODE_847_length_17049_cov_12.9829',
  'NODE_7343_length_4249_cov_22.2454',
  'NODE_19673_length_1550_cov_34.6107',
  'NODE_1409_length_12728_cov_15.5927',
  'NODE_1574_length_11923_cov_14.5027',
  'NODE_711_length_18631_cov_14.7806',
  'NODE_15445_length_2031_cov_23.9084',
  'NODE_15112_length_2083_cov_16.6144',
  'NODE_12470_length_2567_cov_17.4088',
  'NODE_34064_length_837_cov_18.9655',
  'NODE_4057_length_6629_cov_27.157',
  'NODE_865_length_16875_cov_12.6059',
  'NODE_869_length_16855_cov_12.4798',
  'NODE_6477_length_4696_cov_21.0899',
  'NODE_17368_length_1781_cov_15.3621',
  'NODE_18418_length_1671_cov_21.013',
  'NODE_352_length_29307_cov_12.7034',
  'NODE_31105_length_924_cov_16.8895',
  'NODE_4114_length_6570_cov_18.8181',
  'NODE_64132_length_412_cov_37.521',


In [8]:
total_ids = sum(len(id) for id in genome_scaffold_ids.values())
total_ids

280

### Extracting scaffolds with the viral homologs found by HMMER (ids_list), scaffolds shorter than 18000 nt are discarded

#### ids_list_scaffolds = all viral scaffolds above inclusion threshold (may be shorter than 20000)

## to be able to extract the scaffolds, blastable databases need to be created first (for genome and orfs)

In [ ]:
def makeblastdb(file_path, db_type, blastables_outdir):
    db_name = f"{blastables_outdir}/{file_path.split('/')[-1].split('.')[0]}"
    command = ["makeblastdb", "-in", file_path, "-dbtype", db_type, "-out", db_name, "-parse_seqids"]
    subprocess.run(command, check=True)

blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"


for genome_path in genomes:
    makeblastdb(genome_path, "nucl", blastables_outdir) # misto file_path volame s genome_path

for orf_file_path in orf_files:
    makeblastdb(orf_file_path, "prot", blastables_outdir)


### modifying a dictionary for blastdbcmd
#### genome_scaffold_orfs_descr contains the same IDs as genome_scaffold_ids + description

In [9]:
# creating a dict of lists (genomes: ids)
# this dict contains also the description - it allows the usage of the -range option of blastdbcmd command
genome_scaffold_orfs_descr = {}

for file in glob("/home/majnusova/all/projects/plv/data/eustigs_hmmout/*_atp.hmmout"):
    genome_name = os.path.basename(file).split('_atp.hmmout')[0] # getting the real names of the genomes; os.path.basename(file) extracts the base name from the file's full path.
    if genome_name not in genome_scaffold_orfs_descr:
        genome_scaffold_orfs_descr[genome_name] = []

    hmmer_file = SearchIO.read(file, "hmmer3-text")
    
    for record in hmmer_file:
        if record.is_included:
            genome_scaffold_orfs_descr[genome_name].append(record.id + ";" + record.description)

In [10]:
genome_scaffold_orfs_descr

{'Chic10_scaffolds': ['NODE_5080_length_5696_cov_26.6876_1;[649 - 1389] No definition line found',
  'NODE_381_length_27838_cov_14.2215_25;[7239 - 6505] (REVERSE SENSE) No definition line found',
  'NODE_7293_length_4272_cov_22.9291_3;[1285 - 551] (REVERSE SENSE) No definition line found',
  'NODE_5397_length_5445_cov_37.4879_2;[929 - 1711] No definition line found',
  'NODE_6066_length_4954_cov_14.3046_4;[1701 - 964] (REVERSE SENSE) No definition line found',
  'NODE_847_length_17049_cov_12.9829_3;[16205 - 15435] (REVERSE SENSE) No definition line found',
  'NODE_7343_length_4249_cov_22.2454_2;[3283 - 4017] No definition line found',
  'NODE_19673_length_1550_cov_34.6107_1;[1091 - 348] (REVERSE SENSE) No definition line found',
  'NODE_1409_length_12728_cov_15.5927_2;[12037 - 11291] (REVERSE SENSE) No definition line found',
  'NODE_1574_length_11923_cov_14.5027_1;[11636 - 10887] (REVERSE SENSE) No definition line found',
  'NODE_711_length_18631_cov_14.7806_4;[11717 - 12442] No defin

In [11]:
total_ids = sum(len(id) for id in genome_scaffold_orfs_descr.values())
total_ids

280

### Extracting scaffolds (30000nt downstream and upstream of the viral genes) containing viral genes detected by HMMER. 
### Scaffolds shorter than 20000 nts are discarded.

In [ ]:
scaffolds_of_interest = []

def run_blastdbcmd(id, genome_db, outscaffold, range):
    command = ["blastdbcmd", "-entry", str(id), "-db", genome_db, "-out", outscaffold, "-range", range]
    print(f"Running command: {' '.join(command)}")  

    result = subprocess.run(command, text=True, capture_output=True)

    if result.returncode != 0:
        print(f"Warning: blastdbcmd failed for ID {id} in database {genome_db}. Error: {result.stderr}")
        if os.path.exists(outscaffold):
            os.remove(outscaffold)
        return False, None

    if os.path.exists(outscaffold) and os.path.getsize(outscaffold) > 0:
        record = SeqIO.read(outscaffold, "fasta")
        if len(record.seq) < 20000:
            print(f"Notice: Scaffold {id} is shorter than 20000 nucleotides, removing file.")
            os.remove(outscaffold)
            return False, None
        else:
            scaffolds_of_interest.append(id)
            return True, len(record.seq)
    else:
        print(f"Notice: No scaffold file created for ID {id}.")
        return False, None

scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
prot_database = "_orfs."

for genome, ids in genome_scaffold_orfs_descr.items(): #ids = NODE_15445_length_2031_cov_23.9084_2;[1007 - 1726] No definition line found
    print(f"Processing genome: {genome}")  
    genome_dir = os.path.join(scaffolds_dir, genome)
    os.makedirs(genome_dir, exist_ok=True)
    for i in ids:
        res_scaf_orf = i.split(";")[0]
        id_full, descr = i.split(';')
        id = id_full.rsplit('_', 1)[0]  
        print(f"Processing ID: {id}")  

        if "REVERSE" in descr:
            start_orf_part = descr.split(" - ")[1]
            start_orf = start_orf_part.split("]")[0]
            end_orf_part = descr.split(" - ")[0]
            end_orf = end_orf_part.split("[")[1]
        else:
            start_orf_part = descr.split(" - ")[0]
            start_orf = start_orf_part.split("[")[1]
            end_orf_part = descr.split(" - ")[1]
            end_orf = end_orf_part.split("]")[0]

        end_extended_orf = int(end_orf) + 30000 # when a range passed to blastdbcmd exceeds the length of the scaffold, blastdbcmd will by default extract the whole scaffold! :-)
        if int(start_orf) <= 30000:
            start_extended_orf = 1
        else:
            start_extended_orf = int(start_orf) - 30000

        for filename in os.listdir(blastables_outdir):
            if prot_database not in filename:
                base_name = filename.split('.')[0]
                genome_db = os.path.join(blastables_outdir, base_name)
                outscaffold = f"{genome_dir}/{res_scaf_orf}.fasta"

                success, scaffold_length = run_blastdbcmd(id, genome_db, outscaffold, f"{start_extended_orf}-{end_extended_orf}")
                if success:
                    break

print('Scaffolds of interest:', scaffolds_of_interest)


### list containing IDs of scaffolds (not orfs) - input for blastn

In [12]:
#sedi, v poradku; jen ids skafoldu nad inclusion treshold a 20000+
len(scaffolds_of_interest)

NameError: name 'scaffolds_of_interest' is not defined

In [ ]:
#len(scaffolds_of_interest)
scaffolds_of_interest

In [14]:
scaffolds_of_interest= ['NODE_381_length_27838_cov_14.2215',
 'NODE_352_length_29307_cov_12.7034',
 'NODE_45_length_194274_cov_4.45832',
 'NODE_16_length_371769_cov_13.539',
 'NODE_1_length_1589435_cov_13.0886',
 'NODE_10_length_626405_cov_4.34645',
 'NODE_70_length_144148_cov_190.155',
 'NODE_498_length_23434_cov_17.1238',
 'NODE_33_length_241831_cov_197.129',
 'NODE_89_length_113149_cov_3.22244',
 'NODE_33_length_241831_cov_197.129',
 'NODE_146_length_60639_cov_3.37158',
 'NODE_95_length_100189_cov_309.803',
 'NODE_122_length_74444_cov_387.187',
 'NODE_381_length_27838_cov_14.2215',
 'NODE_55_length_165510_cov_190.906',
 'NODE_58_length_162532_cov_4.15186',
 'NODE_98_length_98422_cov_62.5464',
 'NODE_43_length_199121_cov_7.50572',
 'NODE_2_length_969053_cov_7.62238',
 'NODE_37_length_230641_cov_4.40646',
 'NODE_11_length_515168_cov_3.2278',
 'NODE_63_length_153037_cov_4.46219',
 'NODE_244_length_37664_cov_11.8205',
 'NODE_104_length_85964_cov_12.9853',
 'NODE_66_length_150458_cov_3.12336',
 'NODE_3_length_955702_cov_6.81221',
 'NODE_28_length_282822_cov_9.9159',
 'NODE_13_length_427009_cov_190.15',
 'NODE_94_length_101213_cov_13.992',
 'NODE_2_length_969053_cov_7.62238',
 'NODE_95_length_100189_cov_309.803',
 'NODE_3_length_955702_cov_6.81221',
 'NODE_312_length_31651_cov_4.15834',
 'NODE_2_length_969053_cov_7.62238',
 'NODE_34_length_241557_cov_7.5501',
 'NODE_9_length_666414_cov_4.15495',
 'NODE_26_length_312622_cov_7.43917',
 'NODE_1_length_1589435_cov_13.0886',
 'Vischeria_C74_contig_15',
 'Vischeria_C74_contig_12',
 'Vischeria_C74_contig_9',
 'Vischeria_C74_contig_46',
 'Vischeria_C74_contig_4',
 'Vischeria_C74_contig_13',
 'Vischeria_C74_contig_27',
 'Vischeria_C74_contig_13',
 'Vischeria_C74_contig_33',
 'Vischeria_C74_contig_10',
 'Vischeria_C74_contig_10',
 'Vischeria_C74_contig_19',
 'Vischeria_C74_contig_2',
 'Vischeria_C74_contig_4',
 'Vischeria_C74_contig_33',
 'Vischeria_C74_contig_16',
 'Vischeria_C74_contig_25',
 'Vischeria_C74_contig_9',
 'Vischeria_C74_contig_13',
 'Vischeria_C74_contig_9',
 'Vischeria_C74_contig_8',
 'Vischeria_C74_contig_3',
 'Vischeria_C74_contig_18',
 'Vischeria_C74_contig_9',
 'Vischeria_C74_contig_16',
 'Vischeria_C74_contig_12',
 'Vischeria_C74_contig_16',
 'NODE_541_length_101851_cov_46.471309',
 'NODE_55_length_463973_cov_37.984469',
 'NODE_1103_length_61346_cov_37.814180',
 'NODE_320_length_153881_cov_39.008816',
 'NODE_3853_length_25035_cov_37.882483',
 'NODE_4047_length_24047_cov_37.496871',
 'NODE_1923_length_42336_cov_37.970586',
 'NODE_520_length_104572_cov_40.248251',
 'NODE_867_length_72283_cov_40.422873',
 'NODE_4244_length_22945_cov_43.032228',
 'NODE_4874_length_20116_cov_37.345077',
 'NODE_829_length_74096_cov_65.354085',
 'NODE_866_length_72325_cov_37.472691',
 'NODE_1465_length_50215_cov_45.222007',
 'NODE_3667_length_26145_cov_48.502378',
 'NODE_2774_length_32621_cov_36.854996',
 'NODE_4742_length_20636_cov_41.178511',
 'NODE_3601_length_26483_cov_35.616072',
 'NODE_8_length_920040_cov_17.887887',
 'NODE_171_length_238445_cov_572.810663',
 'NODE_311_length_156469_cov_32.611502',
 'NODE_15_length_759953_cov_292.559295',
 'NODE_197_length_219627_cov_52.866117',
 'NODE_152_length_259012_cov_32.238044',
 'NODE_38_length_520036_cov_47.857456',
 'NODE_27_length_600914_cov_14.349319',
 'NODE_179_length_230196_cov_11.871036',
 'NODE_132_length_285120_cov_137.485885',
 'NODE_2364_length_36824_cov_1984.494571',
 'NODE_254_length_184797_cov_7.251294',
 'NODE_377_length_136897_cov_6.550351',
 'NODE_1634_length_47073_cov_31.715572',
 'NODE_3763_length_25504_cov_3.576867',
 'NODE_113_length_322384_cov_46.569383',
 'NODE_4384_length_22242_cov_940.677600',
 'NODE_398_length_128959_cov_17.006463',
 'NODE_619_length_90376_cov_5.509109',
 'NODE_330_length_150702_cov_302.813457',
 'NODE_155_length_252422_cov_128.823373',
 'NODE_395_length_129935_cov_617.085062',
 'NODE_153_length_255781_cov_13.970744',
 'NODE_182_length_228583_cov_79.382664',
 'NODE_48_length_497299_cov_6.449061',
 'NODE_117_length_315731_cov_6.054322',
 'NODE_4297_length_22640_cov_6.481496',
 'NODE_4572_length_21411_cov_2.919518',
 'NODE_965_length_66675_cov_6.078110',
 'NODE_115_length_316723_cov_17.494170',
 'NODE_1_length_1214521_cov_6.930895',
 'NODE_65_length_422212_cov_6.412126',
 'NODE_531_length_103019_cov_5.844194',
 'NODE_173_length_232916_cov_265.596429',
 'NODE_3018_length_30458_cov_4.146045',
 'NODE_35_length_543908_cov_16.984931',
 'NODE_32_length_556671_cov_424.843629',
 'NODE_145_length_273263_cov_31.304994',
 'NODE_741_length_79403_cov_185.477208',
 'NODE_217_length_208486_cov_6.840141',
 'NODE_208_length_212550_cov_43.804855',
 'NODE_34_length_549459_cov_93.351604',
 'NODE_9_length_911479_cov_90.897796',
 'NODE_10_length_908469_cov_12.406104',
 'NODE_39_length_517803_cov_7.286389',
 'NODE_2119_length_39630_cov_2.885243',
 'NODE_1064_length_63035_cov_6.873725',
 'NODE_62_length_444389_cov_18.131093',
 'NODE_135_length_281329_cov_13.280030',
 'NODE_176_length_232184_cov_8.619279',
 'NODE_1_length_1214521_cov_6.930895',
 'NODE_2425_length_36271_cov_3.294192',
 'NODE_157_length_250643_cov_11.185995']

In [15]:
#set protoze potrebujeme scaffold pouzit jako seqidlist v ramci blastn jen jednou
set_scaffolds_of_interest = set(scaffolds_of_interest)
len(set_scaffolds_of_interest)

118

In [16]:
# ted potrebujeme srovnat scaffolds_of_interest s genome_scaffold_ids a pak to pouzijeme pro seqidlist
# ziskame dict: genomes a jejich filtered scaffolds (vice virovych orfu na jednom skafoldu -> redundance)
real_genome_scaffold_ids = {}

for genome, ids in genome_scaffold_orfs_descr.items():
    truncated_ids = [id.split(';')[0] for id in ids] 
    real_genome_scaffold_ids[genome] = truncated_ids

filtered_genome_scaffold_ids = {}

for genome, ids in real_genome_scaffold_ids.items():
    for scaffold_id in ids:
        truncated_id = scaffold_id.split('_')[:-1] 
        truncated_id = '_'.join(truncated_id)
        if truncated_id in set_scaffolds_of_interest:
            if genome not in filtered_genome_scaffold_ids:
                filtered_genome_scaffold_ids[genome] = []  # initialize the list if it doesn't exist
            filtered_genome_scaffold_ids[genome].append(scaffold_id)  

print(filtered_genome_scaffold_ids)

{'Chic10_scaffolds': ['NODE_381_length_27838_cov_14.2215_25', 'NODE_352_length_29307_cov_12.7034_4', 'NODE_45_length_194274_cov_4.45832_227', 'NODE_16_length_371769_cov_13.539_405', 'NODE_1_length_1589435_cov_13.0886_269', 'NODE_10_length_626405_cov_4.34645_166', 'NODE_70_length_144148_cov_190.155_76', 'NODE_498_length_23434_cov_17.1238_3', 'NODE_33_length_241831_cov_197.129_26', 'NODE_89_length_113149_cov_3.22244_46', 'NODE_33_length_241831_cov_197.129_40', 'NODE_146_length_60639_cov_3.37158_5', 'NODE_95_length_100189_cov_309.803_24', 'NODE_122_length_74444_cov_387.187_15', 'NODE_381_length_27838_cov_14.2215_15', 'NODE_55_length_165510_cov_190.906_69', 'NODE_58_length_162532_cov_4.15186_9', 'NODE_98_length_98422_cov_62.5464_9', 'NODE_43_length_199121_cov_7.50572_77', 'NODE_2_length_969053_cov_7.62238_389', 'NODE_37_length_230641_cov_4.40646_312', 'NODE_11_length_515168_cov_3.2278_362', 'NODE_63_length_153037_cov_4.46219_187', 'NODE_244_length_37664_cov_11.8205_8', 'NODE_104_length_859

In [ ]:
sum(len(id) for id in filtered_genome_scaffold_ids.values())

## BLASTn to identify viral repeats - otestovat, jak to bude hledat repetice kdyz se bude blastovat ne proti celemu skafoldu ale proti tem frakcim /home/majnusova/all/projects/plv/data/eve_outscaffolds/

In [ ]:
def blastn_repeats(query, output, database, seqid, outfmt):
    command = ["blastn",
               "-query", str(query), 
               "-out", str(output),
               "-db", str(database), 
               "-strand", "minus",
              # "-dust", "no",
            #   "-word_size", "9",
               "-seqidlist", str(seqid), 
               "-num_threads", "5", # zmenit na clusteru
               "-outfmt", str(outfmt)]
    subprocess.run(command, check=True) 


output_dir = "/home/majnusova/all/projects/plv/data/blastn_tirs/"
blastables_outdir = "/home/majnusova/all/projects/plv/data/blastables/"
path_to_input = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"

for genome, scaffold_ids in filtered_genome_scaffold_ids.items():
    database = f"{blastables_outdir}/{genome}" 
    outdir = os.path.join(output_dir, genome)
    os.makedirs(outdir, exist_ok=True)
    
    for scaffold_id in scaffold_ids:  
        query_file = f"{scaffold_id}.fasta" # scaffold_id = NODE_135_length_281329_cov_13.280030_127 -> NODE_135_length_281329_cov_13.280030_127.fasta
        query = os.path.join(path_to_input, genome, query_file) #/home/majnusova/all/projects/plv/data/eve_outscaffolds/NODE_135_length_281329_cov_13.280030_127.fasta
        seqidlist_file = '_'.join(scaffold_id.split('_')[:-1]) #NODE_135_length_281329_cov_13.280030
        seqidlist_path = os.path.join(outdir, f"{seqidlist_file}_seqid.txt") #/home/majnusova/all/projects/plv/data/alias_test/NODE_135_length_281329_cov_13.280030
        
        with open(seqidlist_path, 'w') as file:
            file.write(seqidlist_file + '\n')  

        output_file = f"{scaffold_id}_repeats.txt"
        output = os.path.join(outdir, output_file)
        output_file2 = f"{scaffold_id}_repeats_human.txt"
        output2 = os.path.join(outdir, output_file2)

    # Checking if the query file exists to avoid the error!
        if os.path.exists(query):
            # outfmt 6 output
            blastn_repeats(query, output, database, seqidlist_path, 5) # should be 5
            # human-readable output
            blastn_repeats(query, output2, database, seqidlist_path, 0)
        else:
            print(f"Query file {query} does not exist, skipping this scaffold ID.")

## Searching for suitable repeats in the blast outputs

In [53]:
import os
from collections import defaultdict
from Bio.Blast import NCBIXML

hsp_info = defaultdict(list)
hsp_tirs_temp = defaultdict(list)  
hsp_tirs_filtered = defaultdict(list) 

blastout_dir = "/home/majnusova/all/projects/plv/data/blastn_tirs/"

for genome in filtered_genome_scaffold_ids.keys():
    fullpath = os.path.join(blastout_dir, genome)
    for blastout in os.listdir(fullpath):
        blastout_path = os.path.join(fullpath, blastout)
        if blastout.endswith("repeats.txt"):
            with open(blastout_path) as blast_handle:
                blast_records = NCBIXML.parse(blast_handle)
                for blast_record in blast_records:
                    for alignment in blast_record.alignments:
                        for hsp in alignment.hsps:
                            if 200 <= len(hsp.match) <= 1200:
                                key = (genome, hsp.bits, hsp.score, hsp.expect, len(hsp.match))  
                                hsp_info[key].append(hsp)
                                tir = hsp.query.replace("-", "")
                                hsp_tirs_temp[(key, blastout)].append((hsp.query_start, tir))  

for (key, blastout), tirs_with_positions in hsp_tirs_temp.items():
    if len(tirs_with_positions) == 2: #interested in only two tirs - more repeats are usually connected to the problems with repeats assembling etc.
        tirs_with_positions.sort() # ascending sorting to avoid negative results
        if tirs_with_positions[1][0] - tirs_with_positions[0][0] >= 17000: # tuple(position, tir), druha[1]-prvni[0] tuple v listu, [1][0] = druha tuple[1] a jeji query_start[0]
            hsp_tirs_filtered[(key, blastout)] = tirs_with_positions


for (key, blastout), tirs in hsp_tirs_filtered.items():
    genome = key[0]
    print(f"genome: {genome}, node: {blastout}, params: {key}, number of TIRs: {len(tirs)}")
    for position, tir in tirs:
        print(f"    start_position: {position}, TIR: {tir}")

# Create a dictionary to store the count of blastout occurrences
blastout_count = {}

# Count the occurrences of each blastout value
for (key, blastout), _ in hsp_tirs_filtered.items():
    if blastout in blastout_count:
        blastout_count[blastout] += 1
    else:
        blastout_count[blastout] = 1

# Filter out non-unique blastout entries from hsp_tirs_filtered
hsp_tirs_filtered = {k: v for k, v in hsp_tirs_filtered.items() if blastout_count[k[1]] == 1}

# Print the filtered dictionary
for (key, blastout), tirs in hsp_tirs_filtered.items():
    genome = key[0]
    print(f"genome: {genome}, node: {blastout}, params: {key}, number of TIRs: {len(tirs)}")
    for position, tir in tirs:
        print(f"    start_position: {position}, TIR: {tir}")

with open("/home/majnusova/all/projects/plv/data/1teststartpositions.txt", "w") as f:
    for (key, blastout), tirs in hsp_tirs_filtered.items():
        f.write(f"{key},{blastout}\n{tirs}\n")


genome: Chic10_scaffolds, node: NODE_104_length_85964_cov_12.9853_33_repeats.txt, params: ('Chic10_scaffolds', 244.878, 132.0, 9.87302e-65, 385), number of TIRs: 2
    start_position: 11332, TIR: TTAACACCCACTCCCCCAGCCCCTCGCCCCGTCGGCGAGGGGAGCCTTTAACAGCCATATTTAGGGGATCGCTCCGGCGACGGTGCTACTGGGGAGCTGTTGCCAACGTGTCGGTACGCCACGGCCCGTGCCAGCGGTTGTGGTGAATACAAACGCGCTGGTGCGGAGCGACTGGCGGGCGCAGCCGCCAGCGTTGGCACACCTATCGCGTCGAAGTCCCACAACCGCAGCCCAGGGCCGTTGAGCGAGCTAAGGCTGGCAACCTCACCCCAACAATGCCCTCTCCGGCAAGCGGAAACAAAATCGGCTGTTAAAGGCTCCCCCTTGCCGACGGGGCGAGGGGGTTGGGGGGAGTGGGTGTTAAAAAAG
    start_position: 57409, TIR: CTTTTTTAACACCCACTCCCCCAACCCCTCGCCCCGTCGGCGAGGGGAGCCTTTTAACAGCCACATCTTGTGGTGTCGCGCCGACTGCATGGCTACGGGGGTGACGTTGCCAGCGTGGGTTGACTCCACGGCCCACGCCTGCGGTTGTGGGTACTTCGGAGAAGGCTGTATGCCAACGCTGGCGGCTGCGCCCGCCAGTGGCTCCGCGCCAATATTTCTTCATTCACTACAACCGCTGGCACTGGCCGTGGCATACCCAACTGTTGGCAGCGGTACTTATGTAGTGCCAAAGCCGGAGTGAAGCACCCAAATGGGCTGTTAAAAGGCTCCCCTCGCCGACGGGGCGAGGGGCTGGGGGAGTGGGTGTTAA
genome: Vischeria_C74_genome_v1

## Turning extracted scaffolds into single line ones to make it possible to search for TIRs in them

In [50]:
len(hsp_tirs_filtered)

12

In [ ]:
scaffolds_dir = "/home/majnusova/all/projects/plv/data/eve_outscaffolds/"
singleline_dir = "/home/majnusova/all/projects/plv/data/eve_singleline_scaffolds/"

for genome, ids in genome_scaffold_orfs_descr.items():
    absolute_path = os.path.join(scaffolds_dir, genome)
    output_genome_dir = os.path.join(singleline_dir, genome)  
    if not os.path.exists(output_genome_dir):
        os.makedirs(output_genome_dir)
    
    for outscaffold in os.listdir(absolute_path):
        outscaffold_path = os.path.join(absolute_path, outscaffold) 
        with open(outscaffold_path, "r") as infile:
            lines = infile.readlines()
        header = lines[0]  # Fasta header
        joined_lines = ''.join(line.strip() for line in lines[1:])  # joining all lines into one except for the first one (header) 
        
        output_file_path = os.path.join(output_genome_dir, outscaffold)
        
        with open(output_file_path, "w") as outfile:
            outfile.write(header) 
            outfile.write(joined_lines)

## Searching for TIRs (saved in hsp_tirs) in single-line scaffolds

In [ ]:
singleline_dir = "/home/majnusova/all/projects/plv/data/eve_singleline_scaffolds/"
tir_dir = "/home/majnusova/all/projects/plv/data/tirs"

if not os.path.exists(tir_dir):
    os.makedirs(tir_dir)

for key_blastout, repeats in hsp_tirs_filtered.items():
    key, blastout = key_blastout[0], key_blastout[1]
    genome = key[0]

    genome_dir = os.path.join(tir_dir, genome)
    if not os.path.exists(genome_dir):
        os.makedirs(genome_dir)
    fasta_filename = blastout.replace("_repeats.txt", ".fasta")
    absolute_path = os.path.join(singleline_dir, genome, fasta_filename)

    if os.path.exists(absolute_path):
        with open(absolute_path, "r") as infile:
            scaffold = infile.read()
            modified = False

            for position, repeat_sequence in repeats:
                if repeat_sequence in scaffold:
                    scaffold = scaffold.replace(repeat_sequence, f"\n\n{repeat_sequence}\n\n")
                    modified = True

            if modified:
                lines = scaffold.split("\n")
                new_scaffold = lines[0]  
                for line in lines[1:]:
                    if line.strip():  
                        new_line = "\n".join([line[i:i+80] for i in range(0, len(line), 80)])  
                        new_scaffold += f"\n{new_line}\n"
                    else:
                        new_scaffold += "\n"

                output_path = os.path.join(genome_dir, fasta_filename)
                with open(output_path, "w") as outfile:
                    outfile.write(new_scaffold)
    else:
        print(f"File not found: {absolute_path}")


## Extracting sequence btw TIRs (gene content) and extracting ORFs using getorf + appending it to the original file

In [ ]:
import os
import subprocess

tir_dir = "/home/majnusova/all/projects/plv/data/tirs"
orfs_dir = "/home/majnusova/all/projects/plv/data/orfs"

if not os.path.exists(orfs_dir):
    os.makedirs(orfs_dir)

def run_getorf(inf, outf):
    command = ["getorf", "-sequence", inf, "-outseq", outf, "-table", "0", "-minsize", "450", "-find", "1"]
    subprocess.run(command, check=True)

for folder in os.listdir(tir_dir):
    folder_path = os.path.join(tir_dir, folder)
    output_folder_path = os.path.join(orfs_dir, folder)

    if not os.path.exists(output_folder_path):
        os.makedirs(output_folder_path)

    for file in os.listdir(folder_path):
        file_path = os.path.join(folder_path, file)
        with open(file_path, "r") as infile:
            scaffold = infile.read()
            sections = scaffold.split("\n\n\n")
            if len(sections) > 2:
                sequence_between_tirs = sections[2].replace("\n", "")
                orf_file_name = os.path.basename(file_path)
                orf_file_path = os.path.join(output_folder_path, f"nucl_{orf_file_name}")

                with open(orf_file_path, "w") as fileorf:
                    fileorf.write(f">{orf_file_name}\n{sequence_between_tirs}\n")

                translated_orf_file_path = os.path.join(output_folder_path, orf_file_name)
                run_getorf(orf_file_path, translated_orf_file_path)

                with open(file_path, "a") as original_file:
                    with open(translated_orf_file_path, "r") as translated_file:
                        original_file.write(translated_file.read())

# HHpred with ORFs against selected databases

## running hhblits to gather homologs of each ORF - adjust paths

## build HMMs from generated a3m files using hmmake

## run hhsearch with generated HMMs (should work similarly to online hhpred)

## prepare blastn to find DRs (shortest one has 48 nt, longest one 101 nt) 